# Algoritmia

## Práctica 2 - Divide y Vencerás

### Curso 2023 - 2024

---

#### Autores:

- César Rodríguez Villagrá

---

Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**:

- Solamente puedes utilizar bibliotecas nativas (https://docs.python.org/es/3.8/library/index.html)
- Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.


In [1]:
#testeable
# imports

In [2]:
#testeable
# Versión ligeramente modificada de la clase Video de la Práctica 1

# Se ha incluido una propiedad 'fee', correspondiente a los costes
# semanales de licenciamiento del contenido
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """

    def __init__(self, name, size, fee):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
        self.fee = fee
        self.espectadores: dict = {}

    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """
        return hash((self.name, self.size, self.fee))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"{self.name}"

    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """
        return str(self)

    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """
        self.espectadores[country] = users

    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """
        self.espectadores.get(country, 0)

    def __ge__(self, other):
        """Muestra si un objeto es mayor o igual que otro

        Parameters
        ----------
        other : Video
            Video a comparar

        Returns
        -------
        Boolean
            True si es mayor o igual que otro, False en caso contrario
        """
        if isinstance(other, Video):
            return self.size >= other.size
        return False

    def __lt__(self, other):
        """Muestra si un objeto es menor que otro

        Parameters
        ----------
        other : Video
            Video a comparar

        Returns
        -------
        Boolean
            True si es menor que otro, False en caso contrario
        """
        if isinstance(other, Video):
            return self.size < other.size
        return False

In [3]:
#testeable

def key_sort(seq, key_function=lambda x: x, reverse=False):
    """Ordena ascendentemente una secuencia utilizando una función clave.

    Parameters
    ----------
    seq : sequence
        Secuencia a ordenar, por ejemplo una lista
    key_function : int
        Función que devuelve para los elementos de la secuencia un valor
        clave para la ordenación
    reverse: boolean
        Si es True Invierte el orden de la ordenación.

    Notes
    -----
    No tiene retorno. Ordena en el sitio la secuencia de entrada.
    """
    def mergeSort(lista: list, prim: int, ult: int) -> None:
        """Implementación del método de ordenación mergeSort en el sitio.

        Parameters
        ----------
        lista : list
            Lista a ordenar
        prim : int
            Índice del primer elemento a ordenar
        ult : int
            Índice del último elemento a ordenar
        """
        if prim < ult:
            medio: int = (prim+ult)//2
            mergeSort(lista, prim, medio)
            mergeSort(lista, medio + 1, ult)
            i: int = prim
            j: int = medio + 1
            temp: list = []
            while i <= medio and j <= ult:
                if key_function(lista[i]) <= key_function(lista[j]):
                    temp.append(lista[i])
                    i += 1
                else:
                    temp.append(lista[j])
                    j += 1
            while i <= medio:
                temp.append(lista[i])
                i += 1
            while j <= ult:
                temp.append(lista[j])
                j += 1
            for pos in range(prim, ult+1):
                lista[pos] = temp[pos - prim]
    mergeSort(seq, 0, len(seq) - 1)
    if reverse == True:
        seq.reverse()

In [4]:
#testeable

def max_emission_profit(video, forecast, ingresos_por_usuario, method="bruteforce"):
    """Calcula el máximo beneficio obtenible emitiendo un vídeo según 
       la previsión de espectadores disponible.

    Parameters
    ----------
    video : Video
        Vídeo a emitir.
    forecast : iterable
        Previsión de espectadores.
    ingresos_por_usuario: float
        Ingresos brutos que proporciona cada usuario.
    method: String
        Algoritmo a emplear 'bruteforce' o 'divideandconquer'.

    Returns
    -------
    float
        Máximo beneficio obtenible.

    Notes
    -----
    El máximo beneficio se calcula considerando la mejor combinación 
    posible de momento de inicio y finalización dentro del periodo de 
    forecast, el coste de licenciamiento del vídeo (fee) y los beneficios 
    obtenidos por cada espectador. Si el vídeo no puede emitirse de forma
    rentable, el método devuelve 0.
    'video' debe disponer de una propiedad 'fee' que establece los costes
    de licenciamiento en periodos equivalentes a las previsiones de 'forecast'.
    """
    def calcularBeneficio(elemento:int) -> float:
        """Calcula el beneficio del índice del elemento pasado.

        Parameters
        ----------
        elemento : int
            Índice de la previsión de espectadores.

        Returns
        -------
        float
            El beneficio correspondiente.
        """
        return forecast[elemento] * ingresos_por_usuario - video.fee

    if method == 'bruteforce':
        maximo: float = 0.0
        for inicio in range(len(forecast)):
            calculo: float = 0.0
            for fin in range(inicio, len(forecast)):
                calculo += calcularBeneficio(fin)
                if calculo > maximo:
                    maximo: float = calculo
        return maximo
    elif method == 'divideandconquer':
        def divYVen(indiceIzq: int, indiceDer: int) -> tuple[float, int, int]:
            """Algoritmo divide y vencerás para calcular el máximo beneficio en un rango de semanas a partir de la previsión de cada semana

            Parameters
            ----------
            indiceIzq : int
                Índice inicial
            indiceDer : int
                Índice final

            Returns
            -------
            tuple[float, int, int]
                Una tupla que contiene el mayor beneficio, junto con los índices de inicio y de fin que generan ese beneficio
            """
            def calcularBeneficioCentro(izq: int, der: int) -> tuple[float, int, int]:
                """Calcula el máximo beneficio la lista de previsiones entre los índices pasados que pase por el centro de ambos. 

                Parameters
                ----------
                izq : int
                    Índice izquierdo
                der : int
                    Índice derecho

                Returns
                -------
                tuple[float, int, int]
                    Una tupla que contiene el mayor beneficio que pasa por el centro, junto con los índices de inicio y de fin que generan ese beneficio
                """
                medio: int = (izq+der)//2
                nuevoIzq = nuevoDer = medio

                maxSumIzq = maxSumDer = -float('inf')
                sumaIzq = sumaDer = 0.0

                for i in range(medio, izq-1, -1):
                    sumaIzq += calcularBeneficio(i)
                    if sumaIzq > maxSumIzq:
                        maxSumIzq: float = sumaIzq
                        nuevoIzq: int = i
                for i in range(medio+1, der+1):
                    sumaDer += calcularBeneficio(i)
                    if sumaDer > maxSumDer:
                        maxSumDer: float = sumaDer
                        nuevoDer: int = i

                return maxSumIzq + maxSumDer, nuevoIzq, nuevoDer

            if indiceIzq == indiceDer:
                return calcularBeneficio(indiceIzq), indiceIzq, indiceDer
            medio: int = (indiceIzq+indiceDer)//2
            benIzq, indiceIzq1, indiceDer1 = divYVen(indiceIzq, medio)
            benDer, indiceIzq2, indiceDer2 = divYVen(medio+1, indiceDer)

            if indiceDer2-indiceIzq1 > 1:
                beneficioCentro: tuple[float, int, int] = calcularBeneficioCentro(
                    indiceIzq1, indiceDer2)
            else:
                beneficioCentro = (benIzq + benDer, indiceIzq1, indiceDer2)

            if (beneficioCentro[0] >= benIzq and beneficioCentro[0] >= benDer):
                return beneficioCentro
            else:
                if benIzq > benDer:
                    return benIzq, indiceIzq1, indiceDer1
                else:
                    return benDer, indiceIzq2, indiceDer2

        if len(forecast) == 0:
            return 0.0
        beneficio: float = divYVen(0, len(forecast)-1)[0]
        if beneficio < 0:
            return 0.0

        return beneficio
    else:
        pass

In [5]:
#testeable
def optimize_content_grid(videos, forecasts, ingresos_por_usuario, k=None):
    """Obtiene los vídeos rentables para su emisión según la previsión
    de espectadores y los ingresos por usuario.

    Parameters
    ----------
    videos : list of Video
        Vídeos a analizar.
    forecasts : list of list of int
        Secuencia de previsiones para los vídeos
    ingresos_por_usuario: float
        Ingresos brutos que proporciona cada usuario.
    k: int
        Obtiene los k elementos más rentables.

    Returns
    -------
    list
        Lista con los k elementos más rentables.

    Notes
    -----
    La rentabilidad de los vídeos se considera tomando el periodo
    óptimo de emisión para cada uno. El método sólo devuelve resultados
    con rentabilidad mayor que 0.
    """

    videosRent: list[Video] = []
    for i, video in enumerate(videos):
        beneficio: float = max_emission_profit(
            video, forecasts[i], ingresos_por_usuario, method='divideandconquer')
        if beneficio > 0:
            videosRent.append((video, beneficio))
    key_sort(videosRent, key_function=lambda x: x[1], reverse=True)

    for i, vid in enumerate(videosRent):
        videosRent[i] = vid[0]

    if k == None:
        return videosRent
    if k >= 0 and k < len(videosRent):
        return videosRent[:k]
    else:
        return videosRent

### Caso de ejemplo


In [6]:
import unittest
import random

# Vídeos de ejemplo para tests
videos_test = [Video("Vikings: Valhalla", 150, 1500),  # Título, tamaño, fee
               Video("New Amsterdam", 56, 980),
               Video("Juego de Tronos", 152, 2500),
               Video("La casa de papel", 76, 800),
               Video("Breaking Bad", 160, 1230),
               Video("Pasión de gavilanes", 125, 350),
               Video("The Sinner", 89, 1900),
               Video("El Cid", 35, 410),
               Video("Raised by wolves", 80, 1900),
               Video("Euphoria", 90, 2000),
               Video("Peacemaker", 40, 1000)]

# Audiencias previstas para los anteriores
forecasts_test = [[500, 1000, 2000,  150, 2000, 1500,  200, 1200,  700],
                  [1000,  800,  500,  250, 4000,  500,  100,  200,  300],
                  [1200, 1000,  900,  900, 1220, 1100,  900, 1000, 1000],
                  [900,  700,  500,  200, 1600,  200,  300,  250,  100],
                  [600,  900, 1240,  250, 2100, 1000,  350, 1000,  800],
                  [190, 1060,  500,  200,  250,  700,  200,  400,  400],
                  [150, 1100, 2500,  600,  800, 3000,  200, 1000,  900],
                  [160,  100,  200,  200,  150,  200,   50,  200,  190],
                  [900,  850,  960,  950, 1100,  100,  350,  900,  800],
                  [600,  900, 1200,  300, 1900, 1000,  650,  850,  800],
                  [510,  500,  500,  500,  500,  500,  500,  500,  510]]


class TestBasico(unittest.TestCase):

    def test_basic_sort(self):
        v = list(range(100))
        random.seed = 1234
        random.shuffle(v)
        key_sort(v)
        self.assertEqual(v, list(range(100)))

    def test_max_profit(self):
        for m in ["bruteforce", "divideandconquer"]:
            self.assertEqual(max_emission_profit(
                Video("test", 99, 1), [1]*10, 1, method=m), 0)
            self.assertEqual(max_emission_profit(
                Video("test", 99, 1), [1]*10, 1, method=m), 0)
            self.assertEqual(max_emission_profit(
                Video("test", 99, 1), [2]*10, 1, method=m), 10)


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de la librería `nbformat`

```bash
$ pip install nbformat
```

###### Explicación de los tests

- `test_ej1_basic_sort`: Comprueba que se ordenen correctamente listas de números.
- `test_ej1_key_sort`: Comprueba que se ordenan correctamente listas según su función clave.
- `test_ej1_video_sort`: Comprueba que se pueden ordenar vídeos.
- `test_ej2_max_profit`: Comprueba que se obtiene el beneficio máximo tanto por fuerza bruta como por divide y vencerás para listas de números simples.
- `test_ej2_max_profit_dataset`: Comprueba que se obtiene el beneficio máximo tanto por fuerza bruta como por divide y vencerás para listas más complejas.
- `test_ej2_optimize_content_grid`: Comprueba el buen desempeño de la función `optimize_content_grid`.

---


### **Informe**

Contesta a las siguientes preguntas.


#### **Complejidad**

1. Método `key_sort`
   - **Complejidad temporal**: Según el teorema maestro, tiene una complejidad de n*Log(n), si la función key_function es O(1) , donde n, es el número de elementos a ordenar, pero si la complejidad de key_function es mayor que O(1), la complejidad de key_sort sería O(n\*O(key_function)).
  Por ejemplo si la complejidad de key_function es O(n), la complejidad de key_sort sería O(n^2), en regla general, si la complejidad de key_function es mayor que O(1), la complejidad sería O(n\*i), donde i es la complejidad de la key_function.
2. Método `max_emission_profit`
   - **Complejidad temporal**: La complejidad de la función es O(n^2), cuando el método es bruteforce,  y O(n\*log(n)), cuando el método es divideandconquer.
3. Método `optimize_content_grid`
   - **Complejidad temporal**: La complejidad de la función es O(n^2\*log(n)), ya que usamos, para cada vídeo de entrada la función max_emission_profit con el método divideandconquer (O(n\*log(n))), por lo que la complejidad es O(n\*n\*log(n))

#### **Ranking de contenidos.**

- ¿Cómo afecta la función clave, que proporciona el criterio de ordenación, a la complejidad temporal del algoritmo?

Tiene una complejidad de n\*Log(n), si la función key_function es O(1) , donde n, es el número de elementos a ordenar, pero si la complejidad de key_function es mayor que O(1), la complejidad de key_sort sería O(n\*O(key_function)).
Esto se debe, a que, aplicando el teorema Maestro, donde la estructura es T(n) = aT(n/b) + O(n^k), en el que en este caso sería de la forma T(n) = 2T(n/2) + n, contando que la key_function es O(1), porque, siendo a=2, b=2, y k=1, como a=b^k, la complejidad es O(n\*log(n)), en el caso que key_function es mayor que O(1), a=2, b=2, y k=i+1, donde i es el grado de la complejidad de key_function, por lo que a < b^k: 2 < 2^(1+i), siendo i > 0, por lo que la complejidad se queda en O(n^k), donde k es mayor que 1.

Por ejemplo si la complejidad de key_function es O(n), la complejidad de key_sort sería O(n^2), en regla general, si la complejidad de key_function es mayor que O(1), la complejidad sería O(n\*i), donde i es la complejidad de la key_function.

#### **Optimización de la compra de contenidos.**

- ¿Has conseguido mejorar la complejidad temporal mediante la solución Divide y Vencerás?
- ¿Por qué ha mejorado? ¿Cuánto ha mejorado? <br>
  <sub>Pon algunos ejemplos numéricos especulativos que relacionen el tamaño del problema con el tiempo de ejecución en ambas versiones. Contrástalos de forma empírica empleando para ello tu implementación.</sub>

Sí, se ha reducido la complejidad de O(n^2), a O(n\*Log(n)), por lo que, cuanto más elementos haya, más diferencia de tiempos habrá, obteniendo ambas el mismo resultado.
En listas muy pequeñas, tendrán tiempos parecidos, pero cada vez serán menos parecidos.

Siguendo las fórmulas de la complejidad, por ejemplo, en una lista de 2 elementos, puede tardar alrededor de 4 unidades el bruteforce, y 2 el de divide y vencerás, siguiendo la fórmula de cada complejidad, con 10 elementos, podría tardar, 100 unidades de tiempo el bruteforce, y alrededor de 33 el divideandconquer, con 100 elementos tardaría alrededor de 10000 unidades y 665 el divideandconquer. Como se ve se puede apreciar que cuanto más grande es la lista, myor es la diferencia en los tiempos, para comprobarlo, he hecho una celda que medirán los tiempos, de ejecución, para el mismo problema, los 2 algoritmos, con cantidades entre 1 y 10000, en el que se van a generar datos aleatorios, para cada iteración, también cada iteración se repite 20 veces para reducir las posibles irregularidades, y muestra la media de esas 20 iteraciones, porque puede haber momentos puntuales en el que el SO le proporcione menos recursos al intérprete de python, y por consecuencia tarde "más de lo debido".

En la salida de la siguiente celda se puede apreciar que la diferencia de tiempos, va del orden de 10^-7, hasta los 3 segundos con 5000 elementos, o incluso 9 segundos con 9000 elementos.

In [7]:
import random
import time

repeticiones: int = 20
tamanos: list[int] = [
    *range(1, 50), *range(50, 1000, 10), *range(1000, 10000, 1000)]
for i in tamanos:
    tiempost1: list[int] = []
    tiempost2: list[int] = []
    for _ in range(repeticiones):
        size:  int = random.randint(1, 200)
        fee: int = random.randint(0, 500)
        vid = Video(f"Prueba{i}-{size}-{fee}", size, fee)
        lista: list[int] = list(range(i))
        random.shuffle(lista)
        ing: int = random.randrange(0, 50)
        tIni1: float = time.perf_counter()
        max_emission_profit(vid, lista, ing, method="bruteforce")
        tFin1: float = time.perf_counter()
        tIni2: float = time.perf_counter()
        max_emission_profit(vid, lista, ing, method="divideandconquer")
        tFin2: float = time.perf_counter()
        t1: float = tFin1-tIni1
        t2: float = tFin2-tIni2
        tiempost1.append(t1)
        tiempost2.append(t2)
    t1 = sum(tiempost1) / len(tiempost1)
    t2 = sum(tiempost2) / len(tiempost2)

    print(f"Tamaño: {i:5d} - Fuerza Bruta: {t1:.2e}s, Divide y Vencerás: {t2:.2e}s, Mejora: {(t1-t2)/t1*100:+9.4f} %, Diferencia de tiempos: {t1-t2:+.3e}")

Tamaño:     1 - Fuerza Bruta: 1.41e-05s, Divide y Vencerás: 3.43e-06s, Mejora:  +75.6824 %, Diferencia de tiempos: +1.067e-05
Tamaño:     2 - Fuerza Bruta: 3.00e-06s, Divide y Vencerás: 4.89e-06s, Mejora:  -63.2722 %, Diferencia de tiempos: -1.895e-06
Tamaño:     3 - Fuerza Bruta: 4.10e-06s, Divide y Vencerás: 7.64e-06s, Mejora:  -86.5684 %, Diferencia de tiempos: -3.545e-06
Tamaño:     4 - Fuerza Bruta: 1.00e-05s, Divide y Vencerás: 9.28e-06s, Mejora:   +7.2463 %, Diferencia de tiempos: +7.250e-07
Tamaño:     5 - Fuerza Bruta: 6.55e-06s, Divide y Vencerás: 1.21e-05s, Mejora:  -84.0582 %, Diferencia de tiempos: -5.510e-06
Tamaño:     6 - Fuerza Bruta: 8.12e-06s, Divide y Vencerás: 1.45e-05s, Mejora:  -78.8924 %, Diferencia de tiempos: -6.410e-06
Tamaño:     7 - Fuerza Bruta: 1.02e-05s, Divide y Vencerás: 2.48e-05s, Mejora: -142.8991 %, Diferencia de tiempos: -1.459e-05
Tamaño:     8 - Fuerza Bruta: 1.28e-05s, Divide y Vencerás: 1.94e-05s, Mejora:  -52.1944 %, Diferencia de tiempos: -6.